In [1]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [2]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import itertools

# Libs de Modelagem e Estatística
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.datasets import get_rdataset
from dieboldmariano import dm_test
import pmdarima as pm
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP, LSTM, Autoformer, NHITS

# Libs de Avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

In [3]:
# =========================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES (SETUP E PROCESSAMENTO)
# =========================================================

In [4]:
def definir_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

In [5]:
def salvar_dataset(serie, dataset_name):
    dir_path = "./data/bronze"
    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, f"{dataset_name.lower()}.csv")
    df = pd.DataFrame({"date": serie.index, "value": serie.values})
    df.to_csv(file_path, index=False)
    print(f"-> Cópia do dataset '{dataset_name}' salva em: {file_path}")

def carregar_serie(nome):
    print(f"Buscando dados de '{nome}' via statsmodels...")
    nome_base = nome.lower()

    if nome_base == "airpassengers":
        df = get_rdataset("AirPassengers", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1949-01-01", periods=len(df), freq="MS"),
                          name="AirPassengers")
    elif nome_base == "lynx":
        df = get_rdataset("lynx", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1821", periods=len(df), freq="A"), name="Lynx")
    elif nome_base == "co2":
        df = get_rdataset("CO2", package="datasets").data
        df = df.ffill()
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1958-03-29", periods=len(df), freq="MS"),
                          name="CO2")
    elif nome_base == "sunspots":
        df = get_rdataset("sunspots", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1749-01-01", periods=len(df), freq="MS"),
                          name="Sunspots")
    elif nome_base == "austres":
        df = get_rdataset("austres", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1971-03-01", periods=len(df), freq="QS-MAR"),
                          name="AustralianResidents")
    elif nome_base == "nottem":
        df = get_rdataset("nottem", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1920-01-01", periods=len(df), freq="MS"),
                          name="Nottingham")
    else:
        raise ValueError(f"Lógica de download para a série '{nome}' não implementada.")

    salvar_dataset(serie, nome)
    return serie

In [6]:
def dividir_serie_temporal(serie, percentual_treino=0.85):
    tamanho_total = len(serie)
    ponto_corte_treino = int(tamanho_total * percentual_treino)
    treino = serie.iloc[:ponto_corte_treino]
    teste = serie.iloc[ponto_corte_treino:]
    return treino, teste

def preparar_dados_para_neuralforecast(serie, nome_serie):
    df = serie.reset_index()
    df.columns = ['ds', 'y']
    df['unique_id'] = nome_serie
    return df

In [7]:
# =========================================================
# SEÇÃO 3: FUNÇÕES PARA CÁLCULO DE MÉTRICAS E MODELAGEM
# =========================================================

In [8]:
def calcular_metricas(y_true, y_pred, y_train):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.inf
    n = len(y_train)
    d = np.sum(np.abs(y_train[1:] - y_train[:-1])) / (n - 1) if n > 1 else np.nan
    mase = np.mean(np.abs(y_true - y_pred)) / d if d is not np.nan and d > 0 else np.inf
    return {'RMSE': rmse, 'MAPE(%)': mape, 'MASE': mase}

In [9]:
# =========================================================
# SEÇÃO 4: PIPELINE AVANÇADO PARA O ARIMA
# =========================================================


In [10]:
def encontrar_melhor_arima_auto(treino_log, freq):
    """Usa auto_arima para encontrar a melhor ordem ARIMA, incluindo sazonalidade."""
    print("Buscando melhor ordem ARIMA com auto_arima...")
    m = 12 if freq.startswith('M') else (4 if freq.startswith('Q') else 1)
    auto_arima_model = pm.auto_arima(treino_log, m=m, seasonal=True, trace=False, error_action='ignore', suppress_warnings=True, stepwise=True)
    print(f"Melhor ordem encontrada: {auto_arima_model.order} Sazonal: {auto_arima_model.seasonal_order}")
    return auto_arima_model.order, auto_arima_model.seasonal_order

In [11]:
# =========================================================
# SEÇÃO 5: PIPELINE DE EXPERIMENTO COMPLETO E AVANÇADO
# =========================================================

In [12]:
def executar_experimento(nome_da_serie, horizonte):
    """Executa o pipeline completo com as correções de estabilidade."""
    try:
        SEED = 42; definir_seed(SEED)
        MAX_INPUT_SIZE = 24; MAX_STEPS_NEURAL = 150
        
        serie_original = carregar_serie(nome_da_serie)
        
        percentual_treino = 1 - (horizonte / len(serie_original))
        if percentual_treino < 0.5: # Garante pelo menos 50% de dados para treino
             print(f"AVISO: Horizonte {horizonte} é muito grande para a série '{nome_da_serie}'. Pulando.")
             return None
        
        treino_orig, teste_orig = dividir_serie_temporal(serie_original, percentual_treino=percentual_treino)
        serie_log = np.log(serie_original)
        treino_log, _ = dividir_serie_temporal(serie_log, percentual_treino=percentual_treino)
        
        freq = serie_original.index.freqstr or pd.infer_freq(serie_original.index)
        if freq is None: return None

        previsoes_teste = {'y_true': teste_orig.values}
        
        # --- 1. Modelo ARIMA ---
        modelo_arima = None
        try:
            print("Processando: ARIMA")
            ordem, ordem_sazonal = encontrar_melhor_arima_auto(treino_log, freq)
            modelo_arima = ARIMA(treino_log.asfreq(freq), order=ordem, seasonal_order=ordem_sazonal).fit()
            # CORREÇÃO: Usando .forecast() para previsão out-of-sample
            preds_log_teste_arima = modelo_arima.forecast(steps=horizonte)
            previsoes_teste['ARIMA'] = np.exp(preds_log_teste_arima).values
        except Exception as e: print(f"AVISO: ARIMA falhou: {e}")

        # --- 2. Modelos Neurais Puros ---
        df_treino_log_nf = preparar_dados_para_neuralforecast(treino_log, nome_da_serie)
        modelos_para_testar = {'N-BEATS': NBEATS, 'MLP': MLP, 'LSTM': LSTM, 'Autoformer': Autoformer, 'NHITS': NHITS}
        
        for nome_modelo, classe_modelo in modelos_para_testar.items():
            try:
                print(f"Processando: {nome_modelo}")
                # CORREÇÃO: Removida a configuração de arquitetura customizada para N-BEATS/NHITS
                modelo_neural = [classe_modelo(input_size=min(2 * horizonte, MAX_INPUT_SIZE), h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf = NeuralForecast(models=modelo_neural, freq=freq)
                nf.fit(df=df_treino_log_nf, verbose=False)
                previsoes_teste[nome_modelo] = np.exp(nf.predict()[classe_modelo.__name__].values)
            except Exception as e: print(f"AVISO: {nome_modelo} falhou: {e}")
        
        # --- 3. Modelo Híbrido (MIMO) ---
        if 'ARIMA' in previsoes_teste and modelo_arima is not None:
            try:
                print("Processando: Híbrido (MIMO)")
                residuos_treino_log = modelo_arima.resid
                df_residuos_nf = preparar_dados_para_neuralforecast(residuos_treino_log, "residuos")
                modelo_residuos = [NBEATS(input_size=min(2*horizonte, MAX_INPUT_SIZE), h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf_residuos = NeuralForecast(models=modelo_residuos, freq=freq)
                nf_residuos.fit(df=df_residuos_nf, verbose=False)
                preds_residuos_log = nf_residuos.predict()['NBEATS'].values
                previsoes_teste['Híbrido (MIMO)'] = previsoes_teste['ARIMA'] + preds_residuos_log
            except Exception as e: print(f"AVISO: Híbrido (MIMO) falhou: {e}")
            
        df_final = pd.DataFrame(previsoes_teste, index=teste_orig.index)
        df_final['dataset'] = nome_da_serie
        df_final['horizonte'] = horizonte
        return df_final.reset_index().rename(columns={'index': 'ds'})
    except Exception as e:
        print(f"ERRO GERAL no processamento de '{nome_da_serie}' para o horizonte {horizonte}: {e}")

In [13]:
# =========================================================
# SEÇÃO 6: ORQUESTRADOR
# =========================================================

In [ ]:
LISTA_DE_DATASETS = ['AirPassengers', 'co2'] 
VETOR_DE_HORIZONTES = [12, 24]
resultados_gerais = []
output_dir = "./data/silver"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "resultados_completos.csv")

for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando Datasets"):
    for horizonte in tqdm(VETOR_DE_HORIZONTES, desc=f"Testando Horizontes para {dataset}", leave=False):
        df_resultado_detalhado = executar_experimento(dataset, horizonte)
        if df_resultado_detalhado is not None:
            resultados_gerais.append(df_resultado_detalhado)

Processando Datasets:   0%|          | 0/2 [00:00<?, ?it/s]

Buscando dados de 'AirPassengers' via statsmodels...
-> Cópia do dataset 'AirPassengers' salva em: ./data/bronze\airpassengers.csv
Processando: ARIMA
Buscando melhor ordem ARIMA com auto_arima...


In [ ]:
if resultados_gerais:
    df_final = pd.concat(resultados_gerais)
    df_final.to_csv(output_file, index=False)
    print(f"\nArquivo '{output_file}' salvo com sucesso!")

In [ ]:
# =========================================================
# SEÇÃO 7: GERAÇÃO DE RELATÓRIOS A PARTIR DOS ARQUIVOS SALVOS
# =========================================================

In [ ]:
print("\n\n" + "="*60)
print("     GERANDO SUÍTE COMPLETA DE RELATÓRIOS")
print("="*60)

try:
    output_file = os.path.join(output_dir, "resultados_completos.csv")
    df_results = pd.read_csv(output_file)
    
    # Extrai os nomes dos modelos das colunas do DataFrame salvo
    modelos = [col for col in df_results.columns if col not in ['ds', 'y_true', 'dataset', 'horizonte']]
    
    # --- 1. Cálculo das Métricas a partir dos resultados brutos ---
    y_train_dict = {}
    for dataset_nome in df_results['dataset'].unique():
        treino, _, _ = dividir_serie_temporal(carregar_serie(dataset_nome))
        y_train_dict[dataset_nome] = treino.values

    # Transforma o DataFrame de formato 'largo' para 'longo' para facilitar o groupby
    df_melted = df_results.melt(id_vars=['ds', 'y_true', 'dataset', 'horizonte'], value_vars=modelos, var_name='Modelo', value_name='y_pred')
    
    metricas_gerais = []
    for (dataset, horizonte, modelo), group in df_melted.groupby(['dataset', 'horizonte', 'Modelo']):
        if not group['y_pred'].isnull().all():
            metricas = calcular_metricas(group['y_true'], group['y_pred'], y_train_dict[dataset])
            metricas['dataset'], metricas['horizonte'], metricas['Modelo'] = dataset, horizonte, modelo
            metricas_gerais.append(metricas)
    
    df_metricas_final = pd.DataFrame(metricas_gerais)
    rename_dict = {'RMSE': 'Mean RMSE', 'MAPE(%)': 'Mean MAPE(%)', 'MASE': 'Mean MASE'}
    df_metricas_final.rename(columns=rename_dict, inplace=True)

    # --- Relatório 1: Evolução do Erro por Horizonte ---
    print("\n--- RELATÓRIO 1: EVOLUÇÃO DO ERRO (RMSE) POR HORIZONTE ---")
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=df_metricas_final, x='horizonte', y='Mean RMSE', hue='Modelo', style='Modelo', markers=True, dashes=False)
    plt.title("Evolução do Erro (RMSE) com o Aumento do Horizonte", fontsize=16)
    plt.xlabel("Horizonte de Previsão"); plt.ylabel("RMSE Médio"); plt.grid(True)
    if not df_metricas_final.empty: plt.xticks(df_metricas_final['horizonte'].unique())
    plt.legend(title='Modelo'); plt.show()
    
    # --- Preparação para os relatórios focados no maior horizonte ---
    maior_horizonte = df_metricas_final['horizonte'].max()
    df_foco_maior_h = df_metricas_final[df_metricas_final['horizonte'] == maior_horizonte]
    display(Markdown(f"### Análises Detalhadas para o Horizonte Mais Longo ({int(maior_horizonte)} passos)"))

    # --- Relatório 2: Desempenho Geral Agregado (foco no maior horizonte) ---
    print("\n--- RELATÓRIO 2: DESEMPENHO GERAL (MÉDIA NO HORIZONTE MAIS LONGO) ---")
    df_agrupado = df_foco_maior_h.groupby('Modelo')[['Mean RMSE', 'Mean MAPE(%)', 'Mean MASE']].mean()
    display(df_agrupado.style.format('{:.3f}').highlight_min(axis=0, props='background-color: #4285F4; color: white;'))

    # --- Relatório 3: Desempenho Detalhado por Dataset (foco no maior horizonte) ---
    print("\n--- RELATÓRIO 3: DESEMPENHO DETALHADO POR DATASET (HORIZONTE MAIS LONGO) ---")
    df_reporte_detalhado = df_foco_maior_h.set_index(['dataset', 'Modelo']).drop(columns=['horizonte'])
    display(df_reporte_detalhado.style.format('{:.3f}'))

    # --- Relatório 4: Tabela de Ranking dos Modelos (foco no maior horizonte) ---
    print("\n--- RELATÓRIO 4: RANKING DOS MODELOS (BASEADO EM RMSE, HORIZONTE MAIS LONGO) ---")
    df_foco_maior_h['Rank'] = df_foco_maior_h.groupby('dataset')['Mean RMSE'].rank().astype(int)
    df_pivot_rank = df_foco_maior_h.pivot_table(index='dataset', columns='Modelo', values='Rank')
    if len(df_pivot_rank) > 1: df_pivot_rank.loc['Média do Rank'] = df_pivot_rank.mean(axis=0)
    display(df_pivot_rank.style.format('{:.1f}').highlight_min(axis=1, props='background-color: #4285F4; color: white;'))
    
    # --- RELATÓRIO 5: TESTE DE HIPÓTESE DIEBOLD-MARIANO (Implementação Corrigida) ---
    print("\n--- RELATÓRIO 5: TESTE DE HIPÓTESE (p-valor, HORIZONTE MAIS LONGO) ---")
    display(Markdown("Comparando cada modelo com o **Híbrido (MIMO)**. Um p-valor < 0.05 (verde) indica que a performance do Híbrido é estatisticamente superior."))
    
    modelo_referencia = 'Híbrido (MIMO)'
    df_teste_maior_h = df_results[df_results['horizonte'] == maior_horizonte]
    dm_results = []

    for dataset_nome, group in df_teste_maior_h.groupby('dataset'):
        if modelo_referencia in group.columns and not group[modelo_referencia].isnull().all():
            row = {'dataset': dataset_nome}
            y_true_reais = group['y_true']
            
            for modelo_competidor in [m for m in modelos if m != modelo_referencia and m in group.columns and not group[m].isnull().all()]:
                # Criar um df temporário para alinhar e remover NaNs do par específico
                temp_df = pd.concat([y_true_reais, group[modelo_referencia], group[modelo_competidor]], axis=1, keys=['y_true', 'ref', 'comp']).dropna()
                
                if not temp_df.empty and len(temp_df) > 1:
                    erros_ref = temp_df['y_true'] - temp_df['ref']
                    erros_comp = temp_df['y_true'] - temp_df['comp']
                    try:
                        _, p_value = dm_test(erros_ref, erros_comp, alternative='less')
                        row[modelo_competidor] = p_value
                    except Exception as e:
                        row[modelo_competidor] = np.nan
                        print(f"AVISO: Teste DM para {modelo_competidor} em {dataset_nome} falhou: {e}")
                else:
                    row[modelo_competidor] = np.nan
            dm_results.append(row)

    if dm_results:
        df_dm = pd.DataFrame(dm_results).set_index('dataset')
        display(df_dm.style.format('{:.3f}').applymap(lambda x: 'background-color: lightgreen; font-weight: bold;' if pd.notna(x) and x < 0.05 else ''))
    else:
        print("Não foi possível gerar o relatório do teste Diebold-Mariano.")

except FileNotFoundError:
    print(f"\nERRO: Arquivo '{output_file}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro ao gerar os relatórios: {e}")